## 1. Introduction

This notebook shows how to connect Jupyter notebooks to a Spark cluster to process data using Spark Python API.

!pip -q install findspark### 2.1. Connection

To connect to the Spark cluster, create a SparkSession object with the following params:

+ **appName:** application name displayed at the [Spark Master Web UI](http://localhost:8080/);
+ **master:** Spark Master URL, same used by Spark Workers;
+ **spark.executor.memory:** must be less than or equals to docker compose SPARK_WORKER_MEMORY config.

In [4]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.8.0 pyspark-shell'

In [5]:
import json, re

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession

jsonSchema = """{'event_id': '33ea832f-869a-4b0d-ad72-3c1966695e88', 'event_timestamp': '2021-06-03 23:57:34.942647', 'event_type': 'pageview', 'page_url': 'http://www.dummywebsite.com/product_b', 'page_url_path': '/product_b', 'referer_url': 'www.bing.com', 'referer_url_scheme': 'http', 'referer_url_port': '80', 'referer_medium': 'search', 'utm_medium': 'organic', 'utm_source': 'bing', 'utm_content': 'ad_2', 'utm_campaign': 'campaign_1', 'click_id': '35a4c0b7-8065-4590-b3d6-eaec7554367b', 'geo_latitude': '44.92801', 'geo_longitude': '4.8951', 'geo_country': 'FR', 'geo_timezone': 'Europe/Paris', 'geo_region_name': 'Valence', 'ip_address': '75.212.102.39', 'browser_name': 'Safari', 'browser_user_agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.2) AppleWebKit/534.22.5 (KHTML, like Gecko) Version/4.0 Safari/534.22.5', 'browser_language': 'lzh_TW', 'os': 'iPad; CPU iPad OS 6_1_6 like Mac OS X', 'os_name': 'iOS', 'os_timezone': 'Europe/Paris', 'device_type': 'Mobile', 'device_is_mobile': "True", 'user_custom_id': 'riveratyler@yahoo.com', 'user_domain_id': '64b35f3d-bdc6-4551-8d8f-bdff5ef202fd'}"""






In [11]:
# Enable SQL commands and Update/Delete/Merge for the current spark session.
# we need to set the following configs
#from delta.tables import DeltaTable

spark = SparkSession.builder \
    .appName("quickstart") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.2,org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.2') \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()


df = spark.read.format("delta").load("/opt/workspace/raw/clickstream/events")
print("Data:\n")
df.show()
print("Schema:\n")
print(df.count())
      







Data:

+----+--------------------+-----+---------+------+--------------------+-------------+
| key|               value|topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----+---------+------+--------------------+-------------+
|null|[7B 22 65 76 65 6...| test|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 65 76 65 6...| test|        0|     1|1969-12-31 23:59:...|            0|
|null|[7B 22 65 76 65 6...| test|        0|     2|1969-12-31 23:59:...|            0|
|null|[7B 22 65 76 65 6...| test|        0|     3|1969-12-31 23:59:...|            0|
|null|[7B 22 65 76 65 6...| test|        0|     4|1969-12-31 23:59:...|            0|
|null|[7B 22 65 76 65 6...| test|        0|     5|1969-12-31 23:59:...|            0|
|null|[7B 22 65 76 65 6...| test|        0|     6|1969-12-31 23:59:...|            0|
|null|[7B 22 65 76 65 6...| test|        0|     7|1969-12-31 23:59:...|            0|
|null|[7B 22 65 76 65 6...| test|        0|    